In [21]:
import cv2
import numpy as np
import yaml

# Bước 1: Đọc file nội tại camera từ cam1.yaml
def load_camera_intrinsics(file_path):
    with open(file_path) as f:
        data = yaml.safe_load(f)
        camera_matrix = np.array(data['camera_matrix']).reshape(3, 3)
        dist_coeffs = np.array(data['dist_coeff']).reshape(-1, 1)
    return camera_matrix, dist_coeffs

# Bước 2: Đọc hình ảnh từ camera
img1 = cv2.imread('left_image.jpg')  # Ảnh từ camera trái
img2 = cv2.imread('right_image.jpg')  # Ảnh từ camera phải



In [22]:

# Load camera intrinsics
camera_matrix, dist_coeffs = load_camera_intrinsics('cam1.yaml')

# Load camera intrinsics
# camera_matrix, dist_coeffs = load_camera_intrinsics('calibration.yaml')




In [23]:
# Bước 3: Tìm điểm đặc trưng (sử dụng ORB)
orb = cv2.ORB_create()
kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)
# Bước 4: Đối sánh điểm đặc trưng
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1, des2)
matches = sorted(matches, key=lambda x: x.distance)
matchespoint= matches.copy()
matche=matchespoint[:30]
# Bước 5: Lấy các điểm đối sánh
pts1 = np.float32([kp1[m.queryIdx].pt for m in matche])
pts2 = np.float32([kp2[m.trainIdx].pt for m in matche])
# Bước 6: Tính ma trận F (Fundamental matrix) và ma trận E (Essential matrix)
F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC)
E = camera_matrix.T @ F @ camera_matrix  # Essential matrix
# Bước 7: Tính R và T từ ma trận E
_, R, T, _ = cv2.recoverPose(E, pts1, pts2, camera_matrix)
baseline = 0.131  # Đơn vị: mét (cần phải biết chính xác khoảng cách này)

In [24]:
# Vẽ các điểm đối sánh
matched_img = cv2.drawMatches(img1, kp1, img2, kp2, matche, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.imshow('Matches', matched_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [25]:
# Bước 6: Tính ma trận F (Fundamental matrix) và ma trận E (Essential matrix)
F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC)
E = camera_matrix.T @ F @ camera_matrix  # Essential matrix
# Bước 7: Tính R và T từ ma trận E
_, R, T, _ = cv2.recoverPose(E, pts1, pts2, camera_matrix)
baseline = 0.131  # Đơn vị: mét (cần phải biết chính xác khoảng cách này)
# Tính disparity
disparities = np.abs(pts1[:, 0] - pts2[:, 0])  # Sự chênh lệch tọa độ x giữa hai ảnh

# Tính khoảng cách Z (Z = f * B / disparity)
focal_length = camera_matrix[0, 0]  # Tiêu cự từ camera intrinsics
depths = (focal_length * baseline) / disparities  # Tính khoảng cách


print(f"Khoảng cách tới vật thể: {sum(depths)/len(depths)} mét")


Khoảng cách tới vật thể: 0.3943174640337626 mét
